In [1]:
from bokeh.plotting import figure
from bokeh.io import output_notebook, show
output_notebook()

from math import sin
import numpy as np
import random

def make_time_series_data(size):
  data = []
  for i in range(size):
    data.append(sin(random.normalvariate(i,0.1)*0.1))
  return np.reshape(np.array(data, dtype=np.float32), (size,1))

def make_batch(data, batch_size, num_steps, num_dimensions, name=None):
  epoch_size =  data.size // (batch_size*num_steps*num_dimensions)
  data = np.lib.stride_tricks.as_strided(
    data, 
    shape=
      (epoch_size,
        batch_size, 
       num_steps+1,
       num_dimensions),
    strides=(
        4*batch_size*num_steps*num_dimensions, 
        4*num_steps*num_dimensions, 
        4*num_dimensions, 
        4 # bytes
    ), 
    writeable=False
  )
    
  return data[:, :, :-1], data[:, :, 1:]


Loading BokehJS ...

In [2]:
import tensorflow as tf

class Model(object):
  def __init__(self, config, is_training=False):
    # config
    self.batch_size = config.batch_size
    self.num_steps = config.num_steps
    self.num_dimensions = config.num_dimensions
    self.keep_prob = config.keep_prob
    self.hidden_size = config.hidden_size
    self.num_layers = config.num_layers
    
    # placeholder
    self.input = tf.placeholder(tf.float32, [None, self.num_steps, self.num_dimensions], name="input")
    self.input_strided = tf.placeholder(tf.float32, [self.batch_size, self.num_steps, self.num_dimensions], name="input_strided")
    self.lr = tf.placeholder(tf.float32, name="learning_rate")
    
    # input layer
    input = tf.reshape(self.input, [-1, self.num_dimensions])
    input_w = tf.get_variable("input_w", [self.num_dimensions, self.hidden_size], dtype=tf.float32)
    input_b = tf.get_variable("input_b", [self.hidden_size], dtype=tf.float32)
    input = tf.nn.xw_plus_b(input, input_w, input_b)
    input = tf.reshape(input, [self.batch_size, self.num_steps, self.hidden_size])
    
    # LSTM layer
    output, state = self._build_rnn_graph(input, is_training)
    
    # output layer
    output = tf.reshape(output, [-1, self.hidden_size])
    output_w = tf.get_variable("output_w", [self.hidden_size, 1], dtype=tf.float32)
    output_b = tf.get_variable("output_b", [1], dtype=tf.float32)
    output = tf.nn.xw_plus_b(output, output_w, output_b)
    self.output = tf.reshape(output, [self.batch_size, self.num_steps, 1])
    
    self.loss = tf.reduce_mean(tf.square(self.output[:, -1, :] - self.input_strided[:, -1, :]))
    self.train_op = tf.train.GradientDescentOptimizer(self.lr).minimize(self.loss, global_step=tf.train.get_or_create_global_step())

  def _build_rnn_graph(self, input, is_training):
    def make_cell():
      cell = tf.contrib.rnn.LSTMBlockCell(
        self.hidden_size, forget_bias=0.0)
      if is_training and self.keep_prob < 1:
        cell = tf.contrib.rnn.DropoutWrapper(
          cell, output_keep_prob=self.keep_prob)
      return cell

    cell = tf.contrib.rnn.MultiRNNCell(
      [make_cell() for _ in range(self.num_layers)], state_is_tuple=True)
    initial_state = cell.zero_state(self.batch_size, tf.float32)
    output, state = tf.nn.dynamic_rnn(cell, input, initial_state=initial_state)
    return output, state

  def learn(self, session, input, input_strided, learning_rate):
    fetches = {
      "loss": self.loss,
      "train_op": self.train_op
    }
    feed_dict = {
      self.input: input,
      self.input_strided: input_strided,
      self.lr: learning_rate
    }
    vals = session.run(fetches, feed_dict=feed_dict)
    loss = vals["loss"]
    return loss

  def predict(self, session, input):
    feed_dict = {
      self.input: np.reshape(input, (1, input.shape[0], input.shape[1]))
    }
    return session.run(self.output, feed_dict=feed_dict)[0,-1]

In [3]:
flags = tf.flags
flags.DEFINE_string("save_path", None,
                    "Model output directory.")
FLAGS = flags.FLAGS

In [4]:
def main(_):

  config = Config()

  with tf.Graph().as_default():
    initializer = tf.random_uniform_initializer(-config.init_scale,
                                                config.init_scale)
    with tf.name_scope("Train"):
      train_input, train_input_strided = make_batch(make_time_series_data(300000), config.batch_size, config.num_steps, config.num_dimensions, name="TrainInput")
      with tf.variable_scope("Model", reuse=None, initializer=initializer):
        m = Model(config=config, is_training=True)
      tf.summary.scalar("Training_Loss", m.loss)
    
    with tf.name_scope("Test"):
      test_input, test_input_strided = make_batch(make_time_series_data(100000), config.batch_size, config.num_steps, config.num_dimensions, name="TestInput")
      with tf.variable_scope("Model", reuse=True, initializer=initializer):
        mtest = Model(config=config)
    
    with tf.variable_scope("Model", reuse=True, initializer=initializer):
        config.batch_size=1
        m2 = Model(config=config)

    sv = tf.train.Supervisor(logdir=FLAGS.save_path, save_summaries_secs=30)
    with sv.managed_session() as session:
      for i in range(train_input.shape[0]):
        lr_decay = config.lr_decay ** max(i + 1 - config.max_epoch, 0.0)
        learning_rate = config.learning_rate * lr_decay
        print("Epoch: %d Learning rate: %.3f" % (i + 1, learning_rate))
        epoch = 0
        loss = m.learn(session, train_input[i], train_input_strided[i], learning_rate)
        print("Training Loss: %.3f" % loss)
      
      losses = 0
      for i in range(test_input.shape[0]):
        loss = m.learn(session, test_input[i], test_input_strided[i], learning_rate)
        losses += loss
      print("Test Loss: %.3f" % (losses / test_input.shape[0]))
    
      predict_term = 100
      data = np.array(make_time_series_data(config.num_steps + predict_term))
      predicted = []
      for i in range(predict_term):
        predicted.append(m2.predict(session, data[i:i+config.num_steps]))
      p = figure(
        title="Hoge", 
        x_axis_label='t', 
        y_axis_label='y')
      p.line(range(predict_term), data[config.num_steps:,0], legend="input")
      p.line(range(predict_term), predicted, legend="predicted", line_color="orange")
      show(p)

    if FLAGS.save_path:
      print("Saving model to %s." % FLAGS.save_path)
      sv.saver.save(session, FLAGS.save_path, global_step=sv.global_step)

In [5]:
class Config(object):
  init_scale = 0.1
  learning_rate = 1.0
  num_layers = 2
  num_steps = 5
  hidden_size = 200
  max_epoch = 3
  batch_size = 3000
  num_dimensions = 1
  keep_prob = 1.0
  lr_decay = 0.5

In [6]:
tf.app.run()

INFO:tensorflow:Starting standard services.
INFO:tensorflow:Starting queue runners.
Epoch: 1 Learning rate: 1.000
Training Loss: 0.504
Epoch: 2 Learning rate: 1.000
Training Loss: 0.456
Epoch: 3 Learning rate: 1.000
Training Loss: 0.401
Epoch: 4 Learning rate: 0.500
Training Loss: 0.319
Epoch: 5 Learning rate: 0.250
Training Loss: 0.193
Epoch: 6 Learning rate: 0.125
Training Loss: 0.125
Epoch: 7 Learning rate: 0.062
Training Loss: 0.104
Epoch: 8 Learning rate: 0.031
Training Loss: 0.094
Epoch: 9 Learning rate: 0.016
Training Loss: 0.090
Epoch: 10 Learning rate: 0.008
Training Loss: 0.088
Epoch: 11 Learning rate: 0.004
Training Loss: 0.087
Epoch: 12 Learning rate: 0.002
Training Loss: 0.087
Epoch: 13 Learning rate: 0.001
Training Loss: 0.086
Epoch: 14 Learning rate: 0.000
Training Loss: 0.086
Epoch: 15 Learning rate: 0.000
Training Loss: 0.086
Epoch: 16 Learning rate: 0.000
Training Loss: 0.086
Epoch: 17 Learning rate: 0.000
Training Loss: 0.086
Epoch: 18 Learning rate: 0.000
Training L

SystemExit: 

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
